# Data Visulization With Soundscapes
![notebook banner image](https://freesound.org/data/displays/156/156031_2703579_wave_L.png "Notebook banner image")
## Introduction 

Museums provide an experience that learning history by reading books doesn’t give you. In a museum, you get to use your senses to take in the history around you, making it feel more intimate, impactful, and ultimately in some cases, more memorable. Using our senses to experience things allows us to connect emotionally with our learning in a way that’s hard to replicate on a page.

This project investigates ways historical information can be presented to leverage the senses to create immersive and impactful experiences. Inspired by the practice of [sonication]( https://programminghistorian.org/en/lessons/sonification), this project will take historical data and transform it into a soundscape, using different foreground and background sounds to create a sense of place. Hopefully, the emotional impact created by the soundscape and the novelty of the whole experience will cause users to reflect on how they view the past. 

Changing how we experience data also has other benefits besides engagement and immersion. Converting data to audio like this makes it accessible to people who may not get the same benefits out of visual data.

Specifically, this project creates a soundscape that simulates an air raid using data from Allied bombing missions during World War II.

---------
## Data 
This project will be using the Theater History of Operations Reports (THOR) dataset of aeriel bombardment operations during World War I, World War II, the Korean War, and the Vietnam War undertaken by the US and other Allied nations. The dataset is available on data.world and can be accessed [here](https://data.world/datamil/world-war-ii-thor-data).

This project uses an abridged version of the dataset that focuses on WW2. This version comes from [this tutorial](https://programminghistorian.org/en/lessons/visualizing-with-bokeh#the-wwii-thor-dataset) courtesy of The Programming Historian. Here is a link to the CSV file of the abridged dataset: [thor_wwii.csv](https://raw.githubusercontent.com/programminghistorian/ph-submissions/gh-pages/assets/visualizing-with-bokeh/thor_wwii.csv)


---------------------------------------------------------------------------------------------------------------------------

## Visualizing The Data

First we need to visualize the dataset so that we can get a better idea of what it looks like. This will help us know if our final soundscape is a decent representation of the data. To do this we will be using [pandas](https://pandas.pydata.org/) to organize our data and [bokeh](https://docs.bokeh.org/en/latest/index.html) to visualize it. 

For our visualization, we will be grouping the data in two-week increments. This returns a dataset that will be a good size for our soundscape.

### Bombings Over Time - Bi-weekly

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
output_notebook()

df = pd.read_csv('data/thor_wwii.csv') #Import the data from CSV into a dataframe

#make sure MSNDATE is a datetime format
df['MSNDATE'] = pd.to_datetime(df['MSNDATE'], format='%m/%d/%Y')

## Here we group the data into a more usable format by aggregating specified time periods together 
 # and specifying what columns we are interested in. The result is a smaller dataframe. 
 ##   
bombs=['TOTAL_TONS', 'TONS_IC', 'TONS_FRAG']
grouped = df.groupby(pd.Grouper(key='MSNDATE', freq='2W'))[bombs].sum() #Grouping the data
grouped = grouped/1000 #Convert to kilatons

source = ColumnDataSource(grouped)# set grouped as the bokeh data source 

p = figure(x_axis_type='datetime')

p.line(x='MSNDATE', y='TOTAL_TONS', line_width=2, source=source, color=Category10[3][0], legend_label='All Munitions')
p.line(x='MSNDATE', y='TONS_FRAG', line_width=2, source=source, color=Category10[3][2], legend_label='Fragmentation')
p.line(x='MSNDATE', y='TONS_IC', line_width=2, source=source, color=Category10[3][1], legend_label='Incendiary')

p.yaxis.axis_label = 'Kilotons of Munitions Dropped'

show(p)

---------------------
## Creating a Soundscape with Scaper

This visual representation of the data is nice and all, but it feels a bit boring and emotionless. Lets try to make this data more engaging and accessible by transforming it into something you can hear. 

For this, I'm going to employ an open-source Python package called [Scaper](https://scaper.readthedocs.io/en/latest/index.html). Scaper allows us to use existing sound files as "source material" to generate soundscapes on the fly. We will feed the data we've analyzed into Scaper to generate a soundscape representation of said data. 

First, we need to install Scaper and its related dependencies. 

In addition to the dependancies added via the code below, we also need a version of [SoX](http://sox.sourceforge.net/), a sound processing program, on our system. There is a version of it included in the project folder of this notebook for convenience. Additional information can be found in the sources at the bottom of this notebook.

Sox **MUST** be installed and added to your PATH. Otherwise, audio generation will not work. A python package called ‘sox’ is included as part of the pip installation below, but it is just a wrapper for python and not SoX itself. (It took me longer than I would like to admit to figure this out...)

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} FFmpeg
!{sys.executable} -m pip install scaper
!{sys.executable} -m pip install playsound



Now that Scaper is installed we need to write a function to create our Scaper object.


In [ ]:
import scaper

def createScaper():
    path_to_audio = 'audio' #Sets location of the audio files
    soundscape_duration = 10.0 #How long the generated soundscape will be, 1o sec is good for this test
    seed = 123 #The seed is used for randomness during generation. We are leaving it static, so we get consistant output
    foreground_folder = 'audio/foreground' #location of foreground audio files (bomb sounds in our case)
    background_folder = 'audio/background' #Location of background audio files
    sc = scaper.Scaper(soundscape_duration, foreground_folder, background_folder) #Actual creaton of the Scaper object
    sc.ref_db = -25  #Setting the volume of the background. Important as all volume is relative to this value.
    
    #Validating that the object was created
    if sc: 
        print("Scaper created successfully")
        return sc
    else:
        print("Error initializing Scaper object")
    
sc = createScaper() #Initialize the Scaper object

Next we need to be able to add background and foreground events to our soundscape. 

We'll start with the background since it won't be dynamic in this basic version of the soundscape. I've chosen an air raid siren that we can have it repeat in the background, as it adds some ambiance and tension.

In [ ]:
sc.add_background(label=('const', 'siren'),
                  source_file=('const', 'audio/background/siren/warningSiren01.wav'), #location of sound
                  source_time=('const', 0)) #Sacper will play the sound starting at this time (so a value of 1 start 1 second into the sound)

Now to add a foreground event. We are going to add a randomly selected volume to this event so that when Scaper generates the soundscape it contains explosions of random different volumes.

In [ ]:
for bombs in range(5):  #This loop will run the code 5 times to generate 5 different bomb events
    sc.add_event(label=('const', 'bomb'), #Catagory (folder) where the sound is located
                 source_file=('const', 'audio/foreground/bomb/blast01.wav'), #Path to the sound
                 source_time=('const', 0), 
                 event_time=('uniform', 0, 9), #Time the sound will play on soundscape. This choses using a uniform distribution of 0-9
                 event_duration=('const', 2.2), #length of time the sound will play
                 snr=('normal', 10, 3), #Volume of the sound relative to the background
                 pitch_shift=(None), #This gives us the option to change the pitch of the sound
                 time_stretch=(None)) ##This gives us the option to stretch the length of the sound

Now all that we need to do is generate our soundscape with the Scaper.generate() function. This will create our soundscape using the random elements we specified. It will also generate a JAMS (JSON Annotated Music Specification) and a text file of the soundscape.  

(Note: I used a static seed here, so each call to generate() will result in the same soundscape. Using a random seed instead would create a unique soundscape each time generate() is called.)

In [ ]:
#Setting name/locations for the generated files
audiofile = 'soundscape.wav'
jamsfile = 'soundscape.jams' 
txtfile = 'soundscape.txt'

sc.generate(audiofile, jamsfile,
            allow_repeated_label=True,
            allow_repeated_source=True,
            reverb=0.1,
            disable_sox_warnings=True,
            no_audio=False, #setting this to true would just generate the text and jams files
            txt_path=txtfile,
            disable_instantiation_warnings=True)

If everything worked like it should a text file containing a textual representation of the newly generated soundscape should have outputted. 

**YAY!** The first soundscape is complete! Please find it in the project folder and give it a listen, or run the code snippet below to listen to it.

### WARNING: CAN BE VERY LOUD, PLAY THE RECORDING WITH CAUTION

In [ ]:
from playsound import playsound # Import the package needed to play sounds from python

# This while loop verifies that the user pants to play the sound. No one likes autoplay.
while 1 == 1:
    userInput = input("Do you want to listen to the soundscape? WARNING: it can be loud. Enter 'y' for yes or 'n' for no: ")
    if userInput == 'y':
        playsound(audiofile)
        break
    elif userInput == 'n':
        break
    else:
        continue

---------------------------------
## Turning Data Into Sound

Now that we've set up both our data modling using pandas and bokeh, and Scaper to manage our soundscape, all that is left to do is to put them together. 

For this, we are going to iterate through the `grouped` data frame we created. We will be iterating through it to use the data from each tuple to generate a unique bomb audio event. This code outputs text to check that the correct data is being accessed. 

In [ ]:
# initialize (so this code can be run seperate from eveyrthing else)
import pandas as pd
df = pd.read_csv('data/thor_wwii.csv')

#make sure MSNDATE is a datetime format
df['MSNDATE'] = pd.to_datetime(df['MSNDATE'], format='%m/%d/%Y')

# group the data
bombs=['TOTAL_TONS', 'TONS_IC', 'TONS_FRAG']
grouped = df.groupby(pd.Grouper(key='MSNDATE', freq='2W'))[bombs].sum()
grouped = grouped/1000 #Convert to kilatons

## loop that iterates each row in grouped. 
 # For now it just outputs the contents of the tuple, followed by a new line with the time and total_tons
for row in grouped.itertuples():
    print(row)
    print(row[0], " - ", row[1]) #prints the key (timestamp) and the first object in the tuple (total_tons)

Now that we can extract the data we need, we need to use it to create our foreground sound events. 

In [ ]:
# First we need to start scaper
import scaper
def createScaper():
    path_to_audio = 'audio'
    soundscape_duration = 170 # We need to change the length to accomadate the new data. 167 items + 3 buffer seconds
    seed = 123
    foreground_folder = 'audio/foreground'
    background_folder = 'audio/background'
    sc = scaper.Scaper(soundscape_duration, foreground_folder, background_folder)
    sc.ref_db = -70 # The reference volume was changed here to allow for a larger difference in explosion volumes
    if sc: 
        print("Scaper created successfully")
        return sc
    else:
        print("Error initializing Scaper object") 
sc = createScaper()

x = 0 #create a variable to keep track of the number of iterations

for row in grouped.itertuples(): #begin our loop
    sc.add_event(label=('const', 'bomb'),
             source_file=('const', 'audio/foreground/bomb/blast01.wav'),
             source_time=('const', 0),
             event_time=('const', x), # This ensures we have one item playing every second (2 weeks war == 1 second soundscape)
             event_duration=('const', 2), # Sounds will overlap a bit
             snr=('const', (row[1]/5)), # Changes the volume of the sound based on the tons of bombs
             pitch_shift=('normal', 2, 1), # pitch shift slightly for some variety 
             time_stretch=(None))
    x += 1 #increment our iterable before looping again
print("Added ",x," bomb explosion sound events")

Now we just need to add our background sound and generate the soundscape.

NOTE: It can take a couple of minutes for it to generate, so be patient. It also uses a lot of memory, especially if run over and over. It might be best to restart the kernel, then run this section (Turning Data Into Sound) on its own. Hopefully, it doesn't crash...  : )

In [ ]:
sc.add_background(label=('const', 'siren'),
                  source_file=('const', 'audio/background/siren/warningSiren01.wav'),
                  source_time=('const', 0))

audiofile = 'bombsOfWW2.wav'
jamsfile = 'bombsOfWW2.jams'
txtfile = 'bombsOfWW2.txt'
sc.generate(audiofile, jamsfile,
            allow_repeated_label=True,
            allow_repeated_source=True,
            reverb=0.1,
            disable_sox_warnings=True,
            no_audio=False,
            txt_path=txtfile,
            disable_instantiation_warnings=True)

**Sucsess!** 

If the generate method was executed correctly, then there should now be a WAV file 2min50sec long titled bombsOfWW2 in the project folder. You can listen to it there, or once again, you could use the code below to give it a listen. I recommend playing it from your project folder as you can control the sound easier, the progress bar helps visualize where you are on the timeline, and there is no way to stop or pause it here. 

Try looking at the bi-weekly graph we made with Bokeh at the beginning and see if you can match up the explosions to the data they represent.

In [ ]:
from playsound import playsound # Import the package needed to play sounds from python

# This while loop verifies that the user pants to play the sound. No one likes autoplay.
while 1 == 1:
    userInput = input("Do you want to listen to the soundscape? WARNING: it can be loud. Enter 'y' for yes or 'n' for no: ")
    if userInput == 'y':
        playsound(audiofile)
        break
    elif userInput == 'n':
        break
    else:
        continue

---------

# Conclusion

There are a bunch of different ways this project could be further developed and improved, for example: 
-	Adding different types of bombs would improve the project immensely. The sounds could be selected based on how many kilotons of explosives were dropped, with the loud ones perhaps shattering glass.
-	Explosions could be separated by type, fragmentation bombs and incendiary bombs having different sounds.
-	The different random elements of the soundscape generation could be modified and tweaked to create a more natural or better-sounding experience.
-	This soundscape could be combined with sonication of the same data as the background.
-	The core idea for this project could also be applied to other data sets, either from other wars or other historical subjects entirely.

As time goes on, people want more interactive and engaging experiences. This project shows that the GLAM sector can use digital methods to create a wide variety of new experiences to meet this demand and present history as a more active, engaging subject, instead of one stuck in the past.


## References
Graham, Shawn. “The Sound of Data (a Gentle Introduction to Sonification for Historians).” Programming Historian, June 7, 2016. https://programminghistorian.org/en/lessons/sonification.

Harper, Charlie. “Visualizing Data with Bokeh and Pandas.” Programming Historian, July 27, 2018. https://programminghistorian.org/en/lessons/visualizing-with-bokeh.

“Installing Python Packages from a Jupyter Notebook | Pythonic Perambulations.” Accessed April 24, 2021. https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/.

Kelly, Lynda. “The (Post) Digital Visitor: What Has (Almost) Twenty Years of Museum Audience Research Revealed?” MW2016: Museums and the Web 2016 (blog), February 16, 2016. https://mw2016.museumsandtheweb.com/paper/the-post-digital-visitor-what-has-almost-20-years-of-museum-audience-research-revealed/.

Marl/Jams. Python. 2014. Reprint, Music and Audio Research Laboratory - NYU, 2021. https://github.com/marl/jams.
“Play Sound in Python - Python Tutorial.” Accessed April 24, 2021. https://pythonbasics.org/python-play-sound/.

Stack Overflow. “Python - How to Iterate over Rows in a DataFrame in Pandas.” Accessed April 24, 2021. https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas.

Salamon, Justin, Duncan MacConnell, Mark Cartwright, Peter Li, and Juan Pablo Bello. “Scaper: A Library for Soundscape Synthesis and Augmentation.” In 2017 IEEE Workshop on Applications of Signal Processing to Audio and Acoustics (WASPAA), 344–48. New Paltz, NY: IEEE, 2017. https://doi.org/10.1109/WASPAA.2017.8170052.

“Scaper — Scaper 1.6.5 Documentation.” Accessed April 24, 2021. https://scaper.readthedocs.io/en/latest/index.html.

“Sox Installation under Windows and Sox Installation in Python - Programmer Sought.” Accessed April 24, 2021. https://www.programmersought.com/article/3710777418/.

data.world. “World War II THOR Data - Dataset by Datamil.” Accessed April 24, 2021. https://data.world/datamil/world-war-ii-thor-data.



## Further Reading
-   For more on sonification check out [this paper](https://programminghistorian.org/en/lessons/sonification) by Shawn Graham on The Programming Historian. 
-   [This paper](https://mw2016.museumsandtheweb.com/paper/the-post-digital-visitor-what-has-almost-20-years-of-museum-audience-research-revealed/) provideds some good insights into the increasing demand for engaging digital experiences amongst museum guests.
-   Last but not least, check out [this paper](http://www.justinsalamon.com/uploads/4/3/9/4/4394963/salamon_scaper_waspaa_2017.pdf) by [Justin Salamon](https://www.justinsalamon.com/) for more infromation on Scaper.
